In [1]:
import torch
import torchvision
from torch import nn 
#from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
import numpy as np
import random
import dataset_utils
from torch.utils.data import Dataset, DataLoader
import tqdm

In [2]:
all_images = torch.load('tensor_dataset/all_images.pt')
all_dnas = torch.load('tensor_dataset/all_dnas.pt')
all_labels = torch.load('tensor_dataset/all_labels.pt')
train_loc = torch.load('tensor_dataset/train_loc.pt')
val_seen_loc = torch.load('tensor_dataset/val_seen_loc.pt')
val_unseen_loc = torch.load('tensor_dataset/val_unseen_loc.pt')
test_seen_loc = torch.load('tensor_dataset/test_seen_loc.pt')
test_unseen_loc = torch.load('tensor_dataset/test_unseen_loc.pt')
species2genus = torch.load('tensor_dataset/species2genus.pt')


In [3]:
all_dnas.unique().shape

torch.Size([2])

In [4]:
dna_train = torch.clone(all_dnas[train_loc].data)
dna_val = torch.clone(torch.cat((all_dnas[val_seen_loc],all_dnas[val_unseen_loc])).data)
dna_test = torch.clone(torch.cat((all_dnas[test_seen_loc],all_dnas[test_unseen_loc])).data)

labels_train = torch.clone(all_labels[train_loc].data)
labels_val = torch.clone(torch.cat((all_labels[val_seen_loc],all_labels[val_unseen_loc])).data)
labels_test = torch.clone(torch.cat((all_labels[test_seen_loc],all_labels[test_unseen_loc])).data)
#un_train = torch.unique(dna_train.view(dna_train.size(0), -1), dim=0)
#un_val = torch.unique(dna_val.view(dna_val.size(0), -1), dim=0)
#un_test = torch.unique(dna_test.view(dna_test.size(0), -1), dim=0)

In [5]:
class DNAdataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.float()
        self.targets = torch.tensor(targets.clone().detach())
        
    def __getitem__(self, index):
        x = self.data[index].unsqueeze(0)
        y = self.targets[index]
        return x, y
    def __len__(self):
        return len(self.data)
        
d_train = DNAdataset(dna_train, labels_train)
d_val = DNAdataset(dna_val, labels_val)
d_train_val = DNAdataset(torch.cat((dna_train,dna_val)), torch.cat((labels_train,labels_val)))
d_test = DNAdataset(dna_test,labels_test)

/tmp/ipykernel_11936/1547906167.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets.clone().detach())


In [6]:
dataloader_train = DataLoader(d_train, batch_size=32,shuffle=True)
dataloader_val = DataLoader(d_val, batch_size=32,shuffle=True)
dataloader_train_val = DataLoader(d_train_val, batch_size=32,shuffle=True)
dataloader_test = DataLoader(d_test, batch_size=32,shuffle=True)
dataloaders = {'train':dataloader_train,'val':dataloader_val,'train_val':dataloader_train_val,'test':dataloader_test}
dataset_sizes = {'train': d_train.data.shape[0], 'val':d_val.data.shape[0],'train_val':d_train_val.data.shape[0],'test':d_test.data.shape[0]}

In [7]:
is_train_val = False # SET TO TRUE IF YOU WANT TO USE TRAIN+VAL FOR TRAINING
if is_train_val:
    dataloaders['train'] = dataloaders['train_val']
    dataloaders['val'] = dataloaders['test']
    dataset_sizes['train'] = dataset_sizes['train_val']
    dataset_sizes['val'] = dataset_sizes['test']

In [8]:

from tqdm.notebook import tqdm
def fit(epochs,dataloaders,optimizer,model,start_idx=0):
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()
    
    train_losses = []
    train_scores = []
    val_losses = []
    val_scores = []
    for epoch in range(epochs):
        running_train_corrects = 0
        for dnas,labels in tqdm(dataloaders['train']):
            model.train()
            dnas = dnas.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            #print(dnas.shape)
            predicted_labels = model(dnas)
            train_loss = criterion(predicted_labels,labels)
            train_loss.backward()
            optimizer.step()
            
            _, preds = torch.max(predicted_labels, 1)
            #print(preds)
            #print(labels.data)
            running_train_corrects += torch.sum(preds == labels.data)
        train_losses.append(train_loss)
        
        running_val_corrects = 0
        for dnas,labels in tqdm(dataloaders['val']):
            
            model.eval()
            with torch.no_grad():
                dnas = dnas.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                predicted_labels = model(dnas)
                val_loss = criterion(predicted_labels,labels)
                
                _, preds = torch.max(predicted_labels, 1)
                #print(preds)
                #print(labels.data)
                running_val_corrects += torch.sum(preds == labels.data)
        val_losses.append(val_loss)
        
        
        
        #real_scores.append(real_score)
        #fit_p.writer.add_scalar('loss_g', loss_g, epoch)
        # Log losses & scores (last batch)
        
        epoch_train_acc = running_train_corrects.double() / dataset_sizes['train']
        epoch_val_acc = running_val_corrects.double() / dataset_sizes['val']
        print("Epoch [{}/{}], train_loss: {:.4f},  train_score: {:.4f},val_loss: {:.4f},  val_score: {:.4f}".format(
            epoch+1, epochs, train_loss, epoch_train_acc,val_loss,epoch_val_acc))
        #print(f"class accuracy real {class_accuracy_real}")
    
    return train_losses

In [9]:
class TinyModel(torch.nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = torch.nn.Conv2d(1,16,(5,1))
        self.activation1 = torch.nn.LeakyReLU()
        self.norm1 = torch.nn.BatchNorm2d(16)
        self.conv2 = torch.nn.Conv2d(16,1,(5,1))
        self.activation2 = torch.nn.LeakyReLU()
        self.norm2 = torch.nn.BatchNorm2d(1)
        self.dropout1= torch.nn.Dropout(0.70)
        self.flat = torch.nn.Flatten()
        self.linear = torch.nn.Linear(3250,1500)
        self.dropout2= torch.nn.Dropout(0.70)
        self.activation3 = torch.nn.LeakyReLU()
        self.linear2 = torch.nn.Linear(1500,1050)
        #self.softmax = torch.nn.Softmax()
    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.activation2(x)
        x = self.norm2(x)
        x = self.dropout1(x)
        x = self.flat(x)
        x = self.linear(x)
        x = self.dropout2(x)
        x = self.activation3(x)
        x = self.linear2(x)
        return x
    def feature_extract(self,x):
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.flat(x)
        return x




tinymodel = TinyModel()
tinymodel.cuda()
 
optimizer = torch.optim.Adam(tinymodel.parameters(),weight_decay=1e-5)
n_params = dataset_utils.count_trainable_parameters(tinymodel);
print(n_params)

6452761


In [23]:
fit(50,dataloaders,optimizer,tinymodel)

  0%|          | 0/408 [00:00<?, ?it/s]

  0%|          | 0/218 [00:00<?, ?it/s]

Epoch [1/50], train_loss: 3.6584,  train_score: 0.1795,val_loss: 6.6115,  val_score: 0.2296


  0%|          | 0/408 [00:00<?, ?it/s]

  0%|          | 0/218 [00:00<?, ?it/s]

...

Epoch [49/50], train_loss: 0.0426,  train_score: 0.9829,val_loss: 5.8326,  val_score: 0.4584


  0%|          | 0/408 [00:00<?, ?it/s]

  0%|          | 0/218 [00:00<?, ?it/s]

Epoch [50/50], train_loss: 0.2047,  train_score: 0.9849,val_loss: 7.4212,  val_score: 0.4585


[tensor(3.6584, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(2.9625, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.5370, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(1.9220, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.8355, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.8697, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.1395, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.2098, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.0271, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.2506, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.4058, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.1509, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.0237, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.3026, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.8947, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(0.0152, device='cuda:0', grad_fn=<NllLossBackward0>),
 tensor(

In [10]:
state_dict = torch.load('checkpoints/CNN_DNA_weights_for_unseen')
tinymodel.load_state_dict(state_dict['model_state_dict'])
optimizer.load_state_dict(state_dict['optimizer_state_dict'])

In [11]:
import scipy.io as io
matdataset = io.loadmat("get_unseen_fresh_samples/unseen_insect_dataset.mat")
all_dnas = torch.tensor(matdataset['all_dnas']).float()
all_genus_labels = matdataset['all_genus_labels'].squeeze()-1
all_dnas.shape


torch.Size([40050, 658, 5])

In [21]:
import scipy.io as io
def new_extract_expanded_dna_features(model : nn.Module,device :str ,save_to_disk : bool = False, save_name_prefix : str = ""):
    class DNAdataset(Dataset):
        def __init__(self, data, targets):
            self.data = torch.tensor(data)
            self.targets = torch.tensor(targets)
            
        def __getitem__(self, index):
            x = self.data[index].unsqueeze(0)
            y = self.targets[index]
            
            
            return x, y
        
        def __len__(self):
            return len(self.data)


    #matdataset = io.loadmat("get_unseen_fresh_samples/unseen_insect_dataset.mat")
    #all_dnas = matdataset['all_dnas']
    #all_genus_labels = matdataset['all_genus_labels'].squeeze()-1
    dataset = DNAdataset(all_dnas, all_genus_labels)
    dataloader= DataLoader(dataset, batch_size=32,shuffle=False)
    print(all_dnas.shape)
    print(all_genus_labels.shape)
    ###actual extraction of the feature from the model
    model.eval()
    with torch.no_grad():
        features = []
        labels = np.array([]) 
        for dnas,batch_labels in dataloader:
            #print(dnas.shape)
            dnas = dnas
            dnas = dnas.to(device)
            fts = model.feature_extract(dnas)
            labels = np.concatenate((labels, batch_labels.cpu().numpy()))
            features.append(fts.cpu().numpy())
            torch.cuda.empty_cache()
        features = torch.tensor(np.concatenate(features))
        labels = torch.tensor(labels)
    return features

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import importlib 
all_dna_features = new_extract_expanded_dna_features(tinymodel,device,
                                               save_to_disk=False)

/tmp/ipykernel_11936/1593543418.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data)


torch.Size([40050, 658, 5])
(40050,)


In [17]:
all_dnas[0].shape

torch.Size([658, 5])

In [29]:
all_dnas[0].unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 658, 5])

In [30]:
tinymodel.eval()
with torch.no_grad():
   print( tinymodel.feature_extract(all_dnas[0].unsqueeze(0).unsqueeze(0).to(device)))

tensor([[ 0.6922, -0.1530,  0.7426,  ...,  0.7426,  0.0632,  0.7426]],
       device='cuda:0')


In [25]:
all_dna_features[0]

tensor([ 0.6922, -0.1530,  0.7426,  ...,  0.7426,  0.0632,  0.7426])

In [57]:
olddataset = io.loadmat('matlab_dataset/insect_dataset.mat')
oldfeatures = olddataset['all_dna_features_cnn_new']
oldgenus_labels = species2genus[olddataset['all_labels']-1].numpy().squeeze()

In [58]:
oldfeatures.shape

(32424, 3250)

In [59]:
oldgenus_labels.shape

(32424,)

In [60]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(oldfeatures, oldgenus_labels)
train_predicted_labels = clf.predict(oldfeatures)
print(f"Training genus accuracy:{np.count_nonzero(train_predicted_labels==oldgenus_labels)/len(oldgenus_labels)}")


Training genus accuracy:0.9999383172958303


In [61]:

val_predicted_labels= clf.predict(all_dna_features)
print(f"Validation genus accuracy:{np.count_nonzero(val_predicted_labels==all_genus_labels)/len(all_genus_labels)}")

Validation genus accuracy:0.2549063670411985


In [62]:
type(all_dna_features)

torch.Tensor

# Save extracted features in .mat

In [63]:
import scipy.io as io
all_dataset = io.loadmat('get_unseen_fresh_samples/unseen_insect_dataset.mat')
all_dataset['all_dna_features_cnn_new'] = all_dna_features.numpy()
io.savemat('get_unseen_fresh_samples/unseen_insect_dataset.mat',all_dataset)



# Random Forest (only used to try features, not indicative of final results)

In [14]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(expanded_train_dna_features,expanded_train_dna_labels )

In [15]:
train_predicted_labels = clf.predict(expanded_train_dna_features)
print(f"Training species accuracy:\
{np.count_nonzero(train_predicted_labels==expanded_train_dna_labels.numpy())/len(expanded_train_dna_labels)}")

Training species accuracy:0.9993864560165657


In [16]:
val_predicted_labels= clf.predict(expanded_val_dna_features)
print(f"Validation species accuracy:{np.count_nonzero(val_predicted_labels==expanded_val_dna_labels.numpy())/len(expanded_val_dna_labels)}")

Validation species accuracy:0.4567936736161035


In [17]:

temp_val_predicted_probs = clf.predict_proba(expanded_val_dna_features)
val_predicted_probs = np.zeros((len(temp_val_predicted_probs),1050))
for i, cls in enumerate(np.arange(1050)):
    if cls in clf.classes_:
        val_predicted_probs[:, cls] = temp_val_predicted_probs[:, list(clf.classes_).index(cls)]
import math
n_correct_genus = 0
for i in range(len(expanded_val_dna_labels)):
    #label_best_specie = val_predicted_probs[i].argmax()
    label_best_specie = val_predicted_labels[i]
    assert(val_predicted_labels[i]==val_predicted_probs[i].argmax())
    #print(label_best_specie.item())
    genus_of_best_species = species2genus[int(label_best_specie.item())]
    #species_same_genus = [k for k,v in species2genus.items() if v == genus_of_best_species]
    #reduced_species = val_predicted_probs[i][species_same_genus]
    #normalized_reduced_species = reduced_species/(reduced_species.sum())
    
    real_genus = species2genus[int(expanded_val_dna_labels[i].item())]
    predicted_genus = genus_of_best_species
    if real_genus == predicted_genus:
        n_correct_genus+=1
print(f"Validation genus accuracy: {n_correct_genus/len(expanded_val_dna_labels)}")


Validation genus accuracy: 0.8046010064701653
